## Goal: Predict Peak Times in the DHBW Cafeteria Using Calendar Data

This notebook integrates and analyzes external calendar data (e.g., public holidays, lecture schedules, exam periods) to improve the prediction of peak hours in the DHBW cafeteria. By identifying patterns in visitor behavior, we aim to support better planning for both cafeteria staff and students.


### Funktion scrape_rapla_calendar

Scrape the Rapla calendar HTML for a given user & file (course).\
Optionally filter calendar week.


In [69]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

def extract_fields(title_text: str) -> dict:
    fields = {
        "veranstaltungstitel": None,
        "sprache": None,
        "studiengang": None,
        "anzahl_studierende": None,
        "geplante_vorlesungsstunden": None,
        "planungsnotiz": None,
        "ressourcen": None,
        "personen": None,
    }

    # Wenn der Titel NICHT mit 'Lehrveranstaltung' beginnt → nimm erste Zeile als Veranstaltungstitel
    if not title_text.strip().lower().startswith("lehrveranstaltung"):
        first_line = title_text.strip().split("\n")[0]
        fields["veranstaltungstitel"] = first_line.strip()
        return fields

    # Andernfalls: Extrahiere strukturierte Informationen
    def extract(label):
        pattern = rf"{label}:\s*\n*(.*?)\n\n"
        match = re.search(pattern, title_text, re.DOTALL | re.IGNORECASE)
        return match.group(1).strip() if match else None

    fields["veranstaltungstitel"] = extract("Titel")
    fields["sprache"] = extract("Sprache")
    fields["studiengang"] = extract("Studiengang")
    fields["anzahl_studierende"] = extract("Anzahl Studierende")
    fields["geplante_vorlesungsstunden"] = extract("Gepl\. Vorlesungsstunden")
    fields["planungsnotiz"] = extract("Planungsnotiz")

    res_match = re.search(r"Ressourcen:\s*\n(.*?)\n\n", title_text, re.DOTALL)
    if res_match:
        ressourcen = res_match.group(1).strip().replace("\n", ",")
        fields["ressourcen"] = ressourcen

    personen_match = re.search(r"Personen:\s*\n(.*?)\n\n", title_text, re.DOTALL)
    if personen_match:
        personen_block = personen_match.group(1).strip()
        personen = list({line.strip() for line in personen_block.split("\n") if line.strip()})
        fields["personen"] = ", ".join(personen)

    return fields


def scrape_rapla_calendar(user: str, file: str, day: int = None, month: int = None, year: int = None):
    base_url = "https://rapla.dhbw-karlsruhe.de/rapla"
    params = {
        "page": "calendar",
        "user": user,
        "file": file
    }

    if day and month and year:
        params.update({
            "day": day,
            "month": month,
            "year": year
        })

    try:
        resp = requests.get(base_url, params=params)
        resp.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(resp.text, "html.parser")
        text = soup.get_text(separator="\n")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()  # Or raise the exception if you want the program to stop

    pattern = re.compile(
        r"(?P<start>\d{2}:\d{2})\s*-\s*(?P<end>\d{2}:\d{2})\s+"
        r"(?P<title>.*?)(?:\n|\s)+"
        r"(?P<weekday>Mo|Di|Mi|Do|Fr|Sa|So)\s+"
        r"(?P<date>\d{2}\.\d{2}\.\d{2})",
        re.DOTALL
    )

    events = []
    for m in pattern.finditer(text):
        try:
            dd, mm, yy = m.group("date").split(".")
            iso_date = f"20{yy}-{mm}-{dd}"
            title_raw = m.group("title").strip()

            extracted = extract_fields(title_raw)

            event = {
                "title": title_raw,
                "start": f"{iso_date} {m.group('start')}",
                "end": f"{iso_date} {m.group('end')}",
                "weekday": m.group("weekday"),
                "raw_date": m.group("date"),
            }
            event.update(extracted)
            events.append(event)
        except Exception as e:
            print(f"Error processing event: {e}, text: {m.group(0)}")
            continue  # Skip to the next event

    df = pd.DataFrame(events)

    if df.empty:
        print("Warning: No events found for this period.")
        return df  # Or handle as appropriate

    try:
        df["start"] = pd.to_datetime(df["start"])
        df["end"] = pd.to_datetime(df["end"])
        df["date"] = df["start"].dt.date
    except KeyError as e:
        print(f"KeyError: {e}.  Columns in DataFrame: {df.columns}")
        return df  # Or handle the error as needed

    return df

In [70]:
import datetime

def get_monday_of_week(year: int, week: int) -> datetime.date:
    # ISO Kalender: Montag als erster Tag der Woche
    # ISO-Woche 1: die Woche mit dem ersten Donnerstag
    return datetime.datetime.strptime(f'{year}-W{week:02d}-1', "%Y-W%W-%w").date()

def scrape_full_year(user: str, file: str, year: int = 2025):
    all_dfs = []
    max_week = 53  # max. KW, 2025 hat 53 Wochen laut ISO-Kalender

    for week in range(1, max_week + 1):
        try:
            monday = get_monday_of_week(year, week)
        except ValueError:
            # Manche Kombinationen können nicht existieren, z.B. KW 53 in manchen Jahren
            continue

        print(f"Scraping Woche {week}, Startdatum {monday}")
        df_week = scrape_rapla_calendar(
            user=user,
            file=file,
            day=monday.day,
            month=monday.month,
            year=monday.year,
        )
        all_dfs.append(df_week)

    # Alle DataFrames zusammenführen
    full_df = pd.concat(all_dfs, ignore_index=True)
    print(f"Gesamter Kalender für {year} gespeichert als kalender_{year}_gesamt.csv")
    return full_df


In [71]:
full_calendar_df = scrape_full_year(user="brockmans", file="WWI22B1", year=2025)
full_calendar_df

Scraping Woche 1, Startdatum 2025-01-06
Scraping Woche 2, Startdatum 2025-01-13
Scraping Woche 3, Startdatum 2025-01-20
Scraping Woche 4, Startdatum 2025-01-27
Scraping Woche 5, Startdatum 2025-02-03
Scraping Woche 6, Startdatum 2025-02-10
Scraping Woche 7, Startdatum 2025-02-17
Scraping Woche 8, Startdatum 2025-02-24
Scraping Woche 9, Startdatum 2025-03-03
Scraping Woche 10, Startdatum 2025-03-10
Scraping Woche 11, Startdatum 2025-03-17
Scraping Woche 12, Startdatum 2025-03-24
Scraping Woche 13, Startdatum 2025-03-31
Scraping Woche 14, Startdatum 2025-04-07
Scraping Woche 15, Startdatum 2025-04-14
Scraping Woche 16, Startdatum 2025-04-21
Scraping Woche 17, Startdatum 2025-04-28
Scraping Woche 18, Startdatum 2025-05-05
Scraping Woche 19, Startdatum 2025-05-12
Scraping Woche 20, Startdatum 2025-05-19
Scraping Woche 21, Startdatum 2025-05-26
Scraping Woche 22, Startdatum 2025-06-02
Scraping Woche 23, Startdatum 2025-06-09
Scraping Woche 24, Startdatum 2025-06-16
Scraping Woche 25, Startd

,title,start,end,weekday,raw_date,veranstaltungstitel,sprache,studiengang,anzahl_studierende,geplante_vorlesungsstunden,planungsnotiz,ressourcen,personen,date
0,"Feiertag\nerstellt am 22.07.24 10:21, zuletzt ...",2025-01-06 08:00:00,2025-01-06 18:00:00,Mo,06.01.25,Feiertag,None,None,None,None,None,None,None,2025-01-06
1,Sonstiger Termin\n\n\nTitel:\n\n\nFeiertag\n\n...,2025-01-07 08:00:00,2025-01-07 18:00:00,Di,07.01.25,Sonstiger Termin,None,None,None,None,None,None,None,2025-01-07
2,Lehrveranstaltung\n\n\nTitel:\n\n\nWahlmodul (...,2025-01-08 08:30:00,2025-01-08 12:45:00,Mi,08.01.25,Wahlmodul (DS/SE): Arbeits- und Organisationsp...,Deutsch,Wirtschaft/Wirtschaftsinformatik,1,25,reserviert von:,"WWI22B1,WWI22B2,A474","Weinsdörfer, Jochen",2025-01-08
3,Lehrveranstaltung\n\n\nTitel:\n\n\nWahlmodul (...,2025-01-07 08:45:00,2025-01-07 13:00:00,Di,07.01.25,Wahlmodul (SC/SE): Aspekte des Datenmanagements,Deutsch,Wirtschaft/Wirtschaftsinformatik,1,25,reserviert von:,"WWI22B5,A471,WWI22B1,WWI22B3,WWI22B2","Nakou, Aikaterini",2025-01-07
4,Lehrveranstaltung\n\n\nTitel:\n\n\nWahlmodul (...,2025-01-09 09:00:00,2025-01-09 13:15:00,Do,09.01.25,Wahlmodul (SC/SE): Aspekte des Datenmanagements,Deutsch,Wirtschaft/Wirtschaftsinformatik,1,25,reserviert von:,"WWI22B5,A471,WWI22B1,WWI22B3,WWI22B2","Nakou, Aikaterini",2025-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,Lehrveranstaltung\n\n\nTitel:\n\n\nSelbststudi...,2025-07-07 08:00:00,2025-07-07 18:00:00,Mo,07.07.25,Selbststudium (Änderungen vorbehalten),Deutsch,Wirtschaft/Wirtschaftsinformatik,1,0,reserviert von:,WWI22B1,None,2025-07-07
152,Lehrveranstaltung\n\n\nTitel:\n\n\nWahlmodul (...,2025-07-08 09:00:00,2025-07-08 12:15:00,Di,08.07.25,Wahlmodul (SC/SE): Weiterführende Aspekte der ...,Deutsch,Wirtschaft/Wirtschaftsinformatik,1,25,reserviert von:,"WWI22B5,WWI22B1,WWI22B3,WWI22B2,A368","Engmann, Noah",2025-07-08
153,Lehrveranstaltung\n\n\nTitel:\n\n\nNeue Konzep...,2025-07-08 09:45:00,2025-07-08 13:00:00,Di,08.07.25,Neue Konzepte I: Data Science und Machine Lear...,Deutsch,Wirtschaft/Wirtschaftsinformatik,1,35,reserviert von:,"WWI22B1,A468","Mecke, Christoph",2025-07-08
154,Hauptklausur IT-Infrastrukturen (150 min)\ners...,2025-07-22 10:00:00,2025-07-22 12:30:00,Di,22.07.25,Hauptklausur IT-Infrastrukturen (150 min),None,None,None,None,None,None,None,2025-07-22
